<a href="https://colab.research.google.com/github/OlanreleOladeji/Demand-forecast--XGBoost/blob/main/Products_quantity_forecasting_model_(XGB).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data upload and preprocessing

In [3]:
import pandas as pd

In [4]:
import numpy as np

# forecast monthly births with xgboost
from numpy import asarray

In [5]:
from google.colab import files

In [6]:
uploaded = files.upload()

Saving 1pharma-data.csv to 1pharma-data.csv


In [7]:
df = pd.read_csv('1pharma-data.csv')

In [8]:
df.head()

,Distributor,Customer Name,City,Country,Latitude,Longitude,Channel,Sub-channel,Product_ID,Product Class,Sales,Price,Revenue,Month,Year,Name of Sales Rep,Manager,Sales Team,Unnamed: 18
0,Gottlieb-Cruickshank,"Zieme, Doyle and Kunze",Lublin,Poland,51.2333,22.5667,Hospital,Private,Topipizole,Mood Stabilizers,4.0,368,1472.0,January,2018,Mary Gerrard,Britanny Bold,Delta,NaN
1,Gottlieb-Cruickshank,Feest PLC,Świecie,Poland,53.4167,18.4333,Pharmacy,Retail,Choriotrisin,Antibiotics,7.0,591,4137.0,January,2018,Jessica Smith,Britanny Bold,Delta,NaN
2,Gottlieb-Cruickshank,Medhurst-Beer Pharmaceutical Limited,Rybnik,Poland,50.0833,18.5000,Pharmacy,Institution,Acantaine,Antibiotics,30.0,66,1980.0,January,2018,Steve Pepple,Tracy Banks,Bravo,NaN
3,Gottlieb-Cruickshank,Barton Ltd Pharma Plc,Czeladź,Poland,50.3333,19.0833,Hospital,Private,Lioletine Refliruvax,Analgesics,6.0,435,2610.0,January,2018,Mary Gerrard,Britanny Bold,Delta,NaN
4,Gottlieb-Cruickshank,Keeling LLC Pharmacy,Olsztyn,Poland,53.7800,20.4942,Pharmacy,Retail,Oxymotroban Fexoformin,Analgesics,20.0,458,9160.0,January,2018,Anne Wu,Britanny Bold,Delta,NaN


Check dataset structure

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254082 entries, 0 to 254081
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Distributor        254082 non-null  object 
 1   Customer Name      254082 non-null  object 
 2   City               254082 non-null  object 
 3   Country            254082 non-null  object 
 4   Latitude           254082 non-null  float64
 5   Longitude          254082 non-null  float64
 6   Channel            254082 non-null  object 
 7   Sub-channel        254082 non-null  object 
 8   Product_ID         254082 non-null  object 
 9   Product Class      254082 non-null  object 
 10  Sales              254082 non-null  float64
 11  Price              254082 non-null  int64  
 12  Revenue            254082 non-null  float64
 13  Month              254082 non-null  object 
 14  Year               254082 non-null  int64  
 15  Name of Sales Rep  254082 non-null  object 
 16  Ma

Data Exploration and Cleaning

Print the column headings

In [10]:
print(df.columns.tolist())


['Distributor', 'Customer Name', 'City', 'Country', 'Latitude', 'Longitude', 'Channel', 'Sub-channel', 'Product_ID', 'Product Class', 'Sales', 'Price', 'Revenue', 'Month', 'Year', 'Name of Sales Rep', 'Manager', 'Sales Team', 'Unnamed: 18']


Dropping columns

In [11]:
df= df.drop(['Channel', 'Sub-channel', 'Product Class', 'Name of Sales Rep', 'Price','Manager', 'Sales Team', 'Customer Name', 'City', 'Country', 'Distributor', 'Latitude', 'Longitude', 'Channel', 'Sub-channel', 'Product Class', 'Unnamed: 18'], axis=1)

In [12]:
df.head()

,Product_ID,Sales,Revenue,Month,Year
0,Topipizole,4.0,1472.0,January,2018
1,Choriotrisin,7.0,4137.0,January,2018
2,Acantaine,30.0,1980.0,January,2018
3,Lioletine Refliruvax,6.0,2610.0,January,2018
4,Oxymotroban Fexoformin,20.0,9160.0,January,2018


Sum the number of items in each month per year

In [13]:
monthly_sales = df.groupby(['Year', 'Month', 'Product_ID'])['Sales'].sum().reset_index()

In [14]:
Monthly_Revenue= df.groupby(['Year', 'Month', 'Product_ID'])['Revenue'].sum().reset_index()

In [15]:
merged_df = pd.merge(monthly_sales, Monthly_Revenue, on=['Year', 'Month', 'Product_ID'])

In [16]:
merged_df.head()

,Year,Month,Product_ID,Sales,Revenue
0,2017,April,Abatatriptan,1410.0,1046220.0
1,2017,April,Abilovir Aprotasol,1974.0,531006.0
2,2017,April,Abobozolid,1350.0,101250.0
3,2017,April,Abranatal Lysoprosate,3767.0,2565327.0
4,2017,April,Abtasol,594.0,447876.0


In [17]:
df= merged_df

In [18]:
df

,Year,Month,Product_ID,Sales,Revenue
0,2017,April,Abatatriptan,1410.0,1046220.0
1,2017,April,Abilovir Aprotasol,1974.0,531006.0
2,2017,April,Abobozolid,1350.0,101250.0
3,2017,April,Abranatal Lysoprosate,3767.0,2565327.0
4,2017,April,Abtasol,594.0,447876.0
...,...,...,...,...,...
11515,2020,September,Zithrotrol,3609.0,494433.0
11516,2020,September,Zonitonin,1300.0,673400.0
11517,2020,September,Zonixamine Immulinum,435.0,129630.0
11518,2020,September,Zynosine,790.0,58460.0


In [19]:
#print(df['Product_ID'].unique())

Feature Engineering

Split Dataset

In [20]:
from sklearn.model_selection import train_test_split
X = df[['Revenue']]
y = df[['Sales']]
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)


In [200]:
train_X

,Revenue
11473,187591.0
782,117680.0
4080,1835708.0
1156,876486.0
1536,174988.0
...,...
11284,1305174.0
5191,404855.0
5390,3921854.0
860,350366.0


In [201]:
train_y

,Sales
11473,1299.0
782,1471.0
4080,2474.0
1156,1322.0
1536,1067.0
...,...
11284,1731.0
5191,2165.0
5390,8563.0
860,1049.0


Dataset Scaling using MinMaxScaler

In [23]:
from sklearn.preprocessing import MinMaxScaler

Scaling the train_X and test_X :

In [24]:
scaler_X = MinMaxScaler()

In [25]:
scaler_X.fit(X)

MinMaxScaler()

In [26]:
train_X_Scaled = scaler_X.transform(train_X)

In [27]:
train_X_Scaled = pd.DataFrame(train_X_Scaled, columns=['Revenue'])

In [28]:
train_X_Scaled.head(10)

,Revenue
0,0.049249
1,0.048355
2,0.070333
3,0.058062
4,0.049088
5,0.051306
6,0.069561
7,0.052523
8,0.047038
9,0.053201


In [29]:
test_X_Scaled = scaler_X.transform(test_X)

In [30]:
test_X_Scaled = pd.DataFrame(test_X_Scaled, columns=['Revenue'])

In [31]:
test_X_Scaled.head(10)

,Revenue
0,0.048843
1,0.055124
2,0.052252
3,0.081784
4,0.055866
5,0.051466
6,0.048486
7,0.072890
8,0.052004
9,0.068085


Scaling the train_y and test_y :

In [32]:
scaler_y = MinMaxScaler()

In [33]:
scaler_y.fit(y)

MinMaxScaler()

In [34]:
train_y_Scaled = scaler_y.transform(train_y)

In [35]:
train_y_Scaled = pd.DataFrame(train_y_Scaled, columns=['Sales'])

In [36]:
train_y_Scaled.head(10)

,Sales
0,0.010522
1,0.011977
2,0.020461
3,0.010717
4,0.008560
5,0.005549
6,0.026788
7,0.005710
8,0.002571
9,0.010734


In [37]:
test_y_Scaled = scaler_y.transform(test_y)

In [141]:
test_y_Scaled = pd.DataFrame(test_y_Scaled, columns=['Sales'])

In [143]:
test_y_Scaled.head(10)

,Sales
0,0.002758
1,0.007418
2,0.005439
3,0.044737
4,0.019539
5,0.006395
6,0.005084
7,0.042581
8,0.022737
9,0.020529


XGBoost Algorithm

In [314]:
from xgboost import XGBRegressor, DMatrix

In [315]:
model = XGBRegressor()

In [316]:
model.fit(train_X_Scaled, train_y_Scaled)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

Make predictions on the test dataset (test_X_Scaled):

In [317]:
forecast = model.predict(test_X_Scaled)

In [318]:
print(forecast)

[0.01352304 0.01482592 0.01230563 ... 0.01165497 0.0124406  0.01148146]


Summarize progress

In [319]:
Progress_summary_df = pd.DataFrame({
        'Actual': test_y_Scaled['Sales'],
         'forecast': forecast[:Actual.shape[0]].flatten()
})

print("Progress summary:")

display(Progress_summary_df.head(5))

Progress summary:


,Actual,forecast
0,0.002758,0.013523
1,0.007418,0.014826
2,0.005439,0.012306
3,0.044737,0.039625
4,0.019539,0.015432


Evaluate model with performance

In [294]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error

In [320]:
mean_squared_error(test_y_Scaled, forecast)

0.0004245069285389036

In [321]:
mean_absolute_error(test_y_Scaled, forecast)

0.008594419807195663

In [322]:
rmse = np.sqrt(mean_squared_error(test_y_Scaled, forecast))
print("RMSE: %f" % (rmse))

RMSE: 0.020604


Inverse transformation

Transform the predictions back to the original scale

In [323]:
forecast_value = scaler_y.inverse_transform(forecast.reshape(-1, 1))

In [324]:
forecast_value

array([[1653.734 ],
       [1807.7645],
       [1509.8086],
       ...,
       [1432.8853],
       [1525.7655],
       [1412.3733]], dtype=float32)

Transform the test_y_Scaled back to the original scale (test_y)

In [325]:
test_y = scaler_y.inverse_transform(test_y_Scaled)

In [326]:
Actual_value = pd.DataFrame(test_y, columns=['Sales'])

In [327]:
Actual_value

,Sales
0,381.0
1,932.0
2,698.0
3,5344.0
4,2365.0
...,...
2299,1104.0
2300,5746.0
2301,3371.0
2302,1460.0


Summary of actual test_y and forecast value

In [328]:
Summary_df = pd.DataFrame({
        'Actual_value': Actual_value['Sales'],
         'forecast_value': forecast_value[:Actual_value.shape[0]].flatten()
})

print("Summary:")

display(Summary_df.head(5))

Summary:


,Actual_value,forecast_value
0,381.0,1653.734009
1,932.0,1807.764526
2,698.0,1509.808594
3,5344.0,4739.577148
4,2365.0,1879.421021


Calculating Root Mean Squared Scaled Error (RMSSE)

Step 1: Find the Mean Squared Error (MSE) of the forecast

In [329]:
forecast_mse = mean_squared_error(Actual_value, forecast)

In [330]:
print(f"forcast_mse: {forecast_mse}")

forcast_mse: 22690848.0


Step 2: Find the Mean Squared Error (MSE) of a one-step naive forecast on the training set.

In [381]:
train_y_mse = np.mean(np.diff(train_y['Sales'])**2)

In [382]:
print(f"train_mse: {train_y_mse}")

train_mse: 24135223.33113799


3:  Calculate the RMSSE

In [387]:
rmsse = np.sqrt(forecast_mse/train_mse)

In [389]:
print(f"RMSSE: {rmsse}")

RMSSE: 0.9696158415688582


In [393]:
def rmsse(train_y, test_y, forecast):
    forecast_mse = mean_squared_error(Actual_value, forecast)
    train_mse = np.mean(np.diff(train_y['Sales'])**2)
    return np.sqrt(forecast_mse/train_mse)

print(f'RMSSE: {rmsse(train_y, test_y, forecast)}')

RMSSE: 0.9696158415688582


In [360]:
import xgboost as xgb

In [371]:
dtrain = xgb.DMatrix(train_X, label=train_y)
dtest = xgb.DMatrix(test_X, label=test_y)

In [372]:
def gradient(forecast_value: np.ndarray, dtrain: xgb.DMatrix) -> np.ndarray:
    '''Compute the gradient squared log error.'''
    y = dtrain.get_label()
    return (np.log1p(forecast_value) - np.log1p(y)) / (forecast_value + 1)

In [373]:
def hessian(forecast_value: np.ndarray, dtrain: xgb.DMatrix) -> np.ndarray:
    '''Compute the hessian for squared log error.'''
    y = dtrain.get_label()
    return ((-np.log1p(forecast_value) + np.log1p(y) + 1) /
            np.power(forecast_value + 1, 2))

In [374]:
def squared_log(forecast_value: np.ndarray, dtrain: xgb.DMatrix) -> Tuple[np.ndarray, np.ndarray]:
    '''Squared Log Error objective. A simplified version for RMSLE used as
    objective function.
    '''

    forecast_value[forecast_value < -1] = -1 + 1e-6
    grad = gradient(forecast_value, dtrain)
    hess = hessian(forecast_value, dtrain)
    return grad, hess

In [375]:
xgb.train({'tree_method': 'hist', 'seed': 1994},  # any other tree method is fine.
           dtrain=dtrain,
           num_boost_round=10,
           obj=squared_log)

In [376]:
def rmsle(forecast_value: np.ndarray, dtrain: xgb.DMatrix) -> Tuple[str, float]:
    '''RMSLE evaluation metric.'''
    y = dtrain.get_label()
    # Ensure non-negative values before applying log1p
    y = np.maximum(0, y)
    forecast_value = np.maximum(0, forecast_value)
    return 'RMSLE', float(np.sqrt(np.mean((np.log1p(forecast_value) - np.log1p(y))**2)))


In [377]:
xgb.train({'tree_method': 'hist', 'seed': 1994,
           'disable_default_eval_metric': 1},
          dtrain=dtrain,
          num_boost_round=10,
          obj=squared_log,
          custom_metric=rmsle,
          evals=[(dtrain, 'dtrain'), (dtest, 'dtest')],
          evals_result=results)

[0]	dtrain-RMSLE:6.88258	dtest-RMSLE:6.87233
[1]	dtrain-RMSLE:6.65184	dtest-RMSLE:6.64171
[2]	dtrain-RMSLE:6.42215	dtest-RMSLE:6.41214
[3]	dtrain-RMSLE:6.19358	dtest-RMSLE:6.18370
[4]	dtrain-RMSLE:5.96623	dtest-RMSLE:5.95649
[5]	dtrain-RMSLE:5.74018	dtest-RMSLE:5.73059
[6]	dtrain-RMSLE:5.51558	dtest-RMSLE:5.50597
[7]	dtrain-RMSLE:5.29248	dtest-RMSLE:5.28288
[8]	dtrain-RMSLE:5.07105	dtest-RMSLE:5.06148
[9]	dtrain-RMSLE:4.85149	dtest-RMSLE:4.84197


Hyperparemeter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    "n_estimators": [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    "learning_rate": [0.01, 0.1],
    "max_depth": [3, 7],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "gamma": [0, 0.1],
   }

In [ ]:
grid_search = GridSearchCV(estimator= XGBRegressor(), param_grid= param_grid, cv=5, verbose=1)

In [ ]:
grid_search.fit(train_X_Scaled, train_y_Scaled)

Fitting 5 folds for each of 320 candidates, totalling 1600 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'colsample_bytree': [0.8, 1.0], 'gamma': [0, 0.1],
                         'learning_rate': [0.01, 0.1], 'max_depth': [3, 7],
                         'n_estimators': [100, 200, 300, 400, 500, 600, 700,
                                          800, 900, 1000],
                         'subsample': [0.8, 1.0]},
             verbose=1)

Best parameters

In [ ]:
grid_search.best_params_

{'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': 3,
 'n_estimators': 100,
 'subsample': 0.8}

Make predictions on the test dataset (test_X_Scaled) using grid search best estimator

In [ ]:
predictions_tuned = y_predict_tuned = grid_search.best_estimator_.predict(test_X_Scaled)

In [ ]:
print(predictions_tuned)

[0.06137752 0.06510728 0.06286702 ... 0.06199182 0.06261616 0.06199182]


Summarize progress after tunning

In [ ]:
Progress_summary_df = pd.DataFrame({
        'Actual': test_y_Scaled['Sales'],
        'Predicted_tuned': predictions_tuned[:test_y_Scaled.shape[0]]
    })

print("Progress summary:")

display(Progress_summary_df.head(5))

Progress summary:


,Actual,Predicted_tuned
0,0.053989,0.061378
1,0.058418,0.065107
2,0.056537,0.062867
3,0.093880,0.089279
4,0.069936,0.065894


Evaluate model with performance after tunning

In [ ]:
mean_squared_error(test_y_Scaled, predictions_tuned)

0.00038102251710370183

In [ ]:
mean_absolute_error(test_y_Scaled, predictions_tuned)

0.007939578965306282

In [ ]:
 root_mean_squared_error(test_y_Scaled, predictions_tuned)

0.019519798457622528